In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import signal
import scipy
import scipy.io as sio

In [ ]:
wavelet_challenge = sio.loadmat('wavelet_codeChallenge.mat')
wavelet_challenge

original = np.squeeze(wavelet_challenge['signal'])
signalFIR = np.squeeze(wavelet_challenge['signalFIR'])
signalMW = np.squeeze(wavelet_challenge['signalMW'])
srate = wavelet_challenge['srate'][0][0]

npnts = len(original)

In [ ]:
# Plot in time domain and frequency domain

time_vec = np.arange(0, npnts)/srate
hz_vec = np.linspace(0, srate, npnts)

# Get amplitude spectra
amp_original = np.abs(scipy.fft.fft(original)/npnts)
amp_signalFIR = np.abs(scipy.fft.fft(signalFIR)/npnts)
amp_signalMW = np.abs(scipy.fft.fft(signalMW)/npnts)

plt.figure()
plt.subplot(311)
plt.plot(time_vec, original, label='original')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (a.u.)')


plt.subplots_adjust(hspace = 1)

plt.subplot(312)
plt.plot(time_vec, signalFIR, label='FIR')
plt.plot(time_vec, signalMW, label='MW')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (a.u.)')
plt.legend()

plt.subplot(313)
plt.plot(hz_vec, amp_original, label='original')
plt.plot(hz_vec, amp_signalFIR, label='FIR')
plt.plot(hz_vec, amp_signalMW, label='MW')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude (a.u.)')
plt.xlim([0,20])
plt.legend()




In [ ]:
# 2-stage wideband filter? 
# Refresh memory on how to do this 

hp_cutoff = 9 # Hz
hp_transw = 1 # Hz
order = int(np.round(11*srate/hp_cutoff))
print(order)

frex = [0, hp_cutoff, hp_cutoff + hp_transw, srate/2]
bshape = [0, 0, 1, 1]

# Make high pass filter
hp_filter = signal.firls(order, bands=frex, desired=bshape, fs=srate)

# Evaluate filter characteristics
plt.subplot(121)
plt.plot(hp_filter)

plt.subplot(122)
hz_hpfilt = np.linspace(0, srate, order)
plt.plot(frex, bshape, label='ideal')
filtpow = np.abs(scipy.fft.fft(hp_filter))**2
plt.plot(hz_hpfilt, filtpow, label='actual')
plt.xlim([0,20])
plt.legend()

In [ ]:
# Apply high pass filter to data
signal_hp = signal.filtfilt(hp_filter, 1, original)

plt.subplot(211)
plt.plot(time_vec, original, label='original')
plt.plot(time_vec, signal_hp, label='high-pass filtered')
plt.legend()
plt.title('Time Domain')

plt.subplots_adjust(hspace = 1)

amp_hp = np.abs(scipy.fft.fft(signal_hp)/npnts)
plt.subplot(212)
plt.plot(hz_vec, amp_original, label='original')
plt.plot(hz_vec, amp_hp, label='high-pass filtered')
plt.plot(hz_vec, amp_signalFIR, label='FIR')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.xlim([0, 20])
plt.title('Frequency Domain')
plt.legend()



In [ ]:
#lowpass filter
lp_cutoff = 14 # Hz
lp_transw = 3 # Hz
order = int(13*srate/lp_cutoff)
print(order)

frex = [0, lp_cutoff, lp_cutoff+lp_transw, srate/2]
bshape = [1, 1, 0, 0]

lp_filter = signal.firls(order, bands=frex, desired=bshape, fs=srate)

plt.subplot(121)
plt.plot(lp_filter)

plt.subplot(122)
hz_filt = np.linspace(0,srate,order)
filtpow = np.abs(scipy.fft.fft(lp_filter))**2 # not dividing by order to make max 1
plt.plot(frex, bshape, label='ideal')
plt.plot(hz_filt, filtpow, label='actual')
plt.xlim([0,20])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')

In [ ]:
# apply low-pass filter to hp_filtered data for 2-stage filter
signal_lp = signal.filtfilt(lp_filter, 1, signal_hp)

# Plot filtered 
plt.subplot(211)
plt.plot(time_vec, signal_hp, label='high pass')
plt.plot(time_vec, signal_lp, label='low pass')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Time Domain')

plt.subplots_adjust(hspace = 1)

amp_lp = np.abs(scipy.fft.fft(signal_lp)/npnts)

plt.subplot(212)
plt.plot(hz_vec, amp_hp, label='high-pass')
plt.plot(hz_vec, amp_lp, label='low-pass')
plt.plot(hz_vec, amp_signalFIR, label='FIR')
plt.legend()
plt.xlim([0, 20])
plt.xlabel('Frequency (Hz)')    
plt.ylabel('Amplitude')
plt.title('Frequency Domain')
